The nested CV is implemented according to the [Raschka, 2018](https://github.com/rasbt/model-eval-article-supplementary/blob/master/code/nested_cv_code.ipynb) and the associated [paper](https://arxiv.org/abs/1811.12808).
The [statistical test](https://gist.github.com/jensdebruijn/13e8eeda85eb8644ac2a4ac4c3b8e732) is implemented according to [Boeckhart and Frank, 2014](https://www.cs.waikato.ac.nz/~eibe/pubs/bouckaert_and_frank.pdf).

In [1]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, RepeatedKFold, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet
from sklearn.svm import LinearSVR
from sklearn.dummy import DummyRegressor

import sklearn
print(f"using sklearn version {sklearn.__version__}")
print(f"using numpy version {np.__version__}")

using sklearn version 0.21.3
using numpy version 1.17.3


In [2]:
# generate dataset
n_samples = 100
n_features = 10
X, y = make_regression(n_samples=n_samples,
                       n_features=n_features,
                       n_informative=3,
                       random_state=42)

# initialize classifiers
elasticnet = ElasticNet(random_state=42)
linearsvr = LinearSVR(random_state=42)
dummyregr = DummyRegressor()

# set up pipelines
pipe_elasticnet = Pipeline([("std", StandardScaler()),
                           ("elasticnet", elasticnet)])
pipe_linearsvr = Pipeline([("std", StandardScaler()),
                           ("linearsvr", linearsvr)])
pipe_dummyregr = Pipeline([("std", StandardScaler()),
                           ("dummyregr", dummyregr)])

# set up parameter grids for optimization (double underscore preceding parameter is important)
# make sure that alpha is > 0.01 (link)
grid_elasticnet = [{"elasticnet__alpha": np.power(2., np.arange(-2, 4)),
                    "elasticnet__l1_ratio": [0, 0.5, 1]}]
grid_linearsvr = [{"linearsvr__C": np.power(10., np.arange(-4, 4))}]
grid_dummyregr = [{"dummyregr__strategy": ["mean", "median"]}]

# set up inner cv of nested cv for parameter tuning
gridcvs = {}
kfolds_inner = 2
inner_nestedcv = KFold(n_splits=kfolds_inner, shuffle=True, random_state=42)

for pgrid, est, name in zip((grid_elasticnet, grid_linearsvr, grid_dummyregr),
                            (pipe_elasticnet, pipe_linearsvr, pipe_dummyregr),
                            ("ElasticNet", "LinearSVR", "Dummy")):
    # refit=True, once optimization is done, refit the entire dataset
    gcv = GridSearchCV(estimator=est,
                       param_grid=pgrid,
                       scoring="neg_mean_squared_error",
                       n_jobs=1,
                       cv=inner_nestedcv,
                       verbose=1,
                       refit=True)
    gridcvs[name] = gcv

In [3]:
repetitions = 2
repetition_seeds = range(repetitions)
kfolds_outer = 5

comparisons = [["ElasticNet", "Dummy"],
               ["LinearSVR", "Dummy"],
               ["ElasticNet", "LinearSVR"]]
               
results_all_comparisons = []

for c in comparisons:
    
    print(f"running comparison of {c[0]} and {c[1]}")
               
    # select the relevant subset from gridcsv
    gridcvs_c = {key: value for key, value in gridcvs.items() if key in c}
    
    # pre-allocate data container for saving results of the current comparison
    results_c = {}
    for i in range(len(c)):
        results_c[c[i]] = np.zeros((repetitions, kfolds_outer))
    
    for j in repetition_seeds:
               
        print(f"starting repetition {j + 1} with seed {j}")

        # set up outer cv of nested for parameter (i.e., model) evaluation; ensure that for
        # all comparisons, the randomization of folds is identical
        outer_nestedcv = KFold(n_splits=kfolds_outer, shuffle=True, random_state=j)

        # per model, optimize the parameters using the inner_nestedcv (run internally
        # in cross_val_score) and evaluate the performance of the best parameter setting
        # on the outer_nestedcv (also run internally in cross_val_score)
        for name, gs_est in sorted(gridcvs_c.items()):
               
            print(f"optimizing and evalutaing {name}")
               
            # gs_est returns an estimator that uses the optimized parameters to
            # predict the labels on the test folds of outer_nestedcv
            cv_score = cross_val_score(gs_est,
                                       X=X,
                                       y=y,
                                       cv=outer_nestedcv,
                                       n_jobs=1)
            print(f"{name} | nested RMSE {cv_score.mean() * 100} +/- {cv_score.std() * 100}") 
            # save the scores of all iterations i of the outer_nestedcv
            results_c[name][j, :] = cv_score
    
    # save the results of the current comparison
    results_all_comparisons.append(results_c)

running comparison of ElasticNet and Dummy
starting repetition 1 with seed 0
optimizing and evalutaing Dummy
Fitting 2 folds for each of 2 candidates, totalling 4 fits
Fitting 2 folds for each of 2 candidates, totalling 4 fits
Fitting 2 folds for each of 2 candidates, totalling 4 fits

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.



Fitting 2 folds for each of 2 candidates, totalling 4 fits
Fitting 2 folds for each of 2 candidates, totalling 4 fits
Dummy | nested RMSE -481902.0592054449 +/- 121624.12474218727
optimizing and evalutaing ElasticNet
Fitting 2 folds for each of 18 candidates, totalling 36 fits


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\JohnDoe\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 15998.916629309182, tolerance: 16.2549128595671
  positive)
C:\Users\JohnDoe\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 19544.097277662462, tolerance: 20.995623730113493
  positive)
C:\Users\Jo

Fitting 2 folds for each of 18 candidates, totalling 36 fits


...

C:\Users\JohnDoe\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 13874.205625489127, tolerance: 14.984468950924919
  positive)
C:\Users\JohnDoe\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 17079.981743200402, tolerance: 17.70272031054237
  positive)
C:\Users\JohnDoe\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 23140.17863216805, tolerance: 14.984468950924919
  positive)
C:\Users\JohnDoe\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. D

Fitting 2 folds for each of 18 candidates, totalling 36 fits


C:\Users\JohnDoe\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 95048.64480494647, tolerance: 22.091770396885504
  positive)
[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


ElasticNet | nested RMSE -19.791551096905316 +/- 2.3265229072133344
optimizing and evalutaing LinearSVR
Fitting 2 folds for each of 8 candidates, totalling 16 fits
Fitting 2 folds for each of 8 candidates, totalling 16 fits
Fitting 2 folds for each of 8 candidates, totalling 16 fits
Fitting 2 folds for each of 8 candidates, totalling 16 fits
Fitting 2 folds for each of 8 candidates, totalling 16 fits
LinearSVR | nested RMSE -3.6554872724119323e-09 +/- 5.65025572759494e-09


[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.0s finished


In [4]:
from scipy.stats import t
from math import sqrt
from statistics import stdev

def corrected_dependent_ttest(data1, data2, n_training_folds, n_test_folds, alpha):
    n = len(data1)
    differences = [abs(data1[i]-data2[i]) for i in range(n)]
    sd = stdev(differences)
    divisor = 1 / n * sum(differences)
    test_training_ratio = n_test_folds / n_training_folds  
    denominator = sqrt(1 / n + test_training_ratio) * sd
    t_stat = divisor / denominator
    # degrees of freedom
    df = n - 1
    # calculate the critical value
    cv = t.ppf(1.0 - alpha, df)
    # calculate the p-value
    p = (1.0 - t.cdf(abs(t_stat), df)) * 2.0
    # return everything
    return t_stat, df, cv, p

In [5]:
# test all comparisons for significance
for result in results_all_comparisons:
    # combine the RMSEs of all folds in one vector of lengts r * k
    for key, value in result.items():
        #print(key, value)
        result[key] = np.ravel(value)
       
    models = list(result.keys())
    scores = list(result.values())
    
    nperfold = n_samples // kfolds_outer
    ntraininginstances = nperfold * (kfolds_outer - 1)
    ntestinstances = nperfold
    alpha = 0.05 / len(comparisons)
    
    t_stat, df, cv, p = corrected_dependent_ttest(scores[0],
                                                  scores[1],
                                                  ntraininginstances,
                                                  ntestinstances,
                                                  alpha)
    print(f"the comparison of {models[0]} and {models[1]} resulted in t = {t_stat}, p = {p}, with a critical value of {cv} and {df} degrees of freedom")
        

the comparison of ElasticNet and Dummy resulted in t = 5.5703920632572075, p = 0.00034720650074504533, with a critical value of 2.5095870571050907 and 9 degrees of freedom
the comparison of LinearSVR and Dummy resulted in t = 5.570583612426464, p = 0.00034712110278323216, with a critical value of 2.5095870571050907 and 9 degrees of freedom
the comparison of ElasticNet and LinearSVR resulted in t = 6.927291159199279, p = 6.85538101048433e-05, with a critical value of 2.5095870571050907 and 9 degrees of freedom


In [6]:
################################################
# self-contained, minimal, reproducible examples
################################################

# repeated k-fold

X, y = make_regression(n_samples=30, n_features=2)

repetitions = 3
repetition_seeds = range(repetitions)

for j in repetition_seeds:
    
    cv = KFold(n_splits=3, random_state=j, shuffle=True)
    
    print(f"repetition {j + 1} with seed {j}")
    
    for trainidx, testidx in cv.split(X):
        print(trainidx, testidx)

repetition 1 with seed 0
[ 0  1  3  4  5  6  7  8  9 12 14 15 16 18 19 20 21 23 25 29] [ 2 10 11 13 17 22 24 26 27 28]
[ 0  2  3  7  9 10 11 12 13 15 17 18 19 21 22 24 25 26 27 28] [ 1  4  5  6  8 14 16 20 23 29]
[ 1  2  4  5  6  8 10 11 13 14 16 17 20 22 23 24 26 27 28 29] [ 0  3  7  9 12 15 18 19 21 25]
repetition 2 with seed 1
[ 0  1  2  4  5  6  7  8  9 11 12 13 15 16 18 23 25 27 28 29] [ 3 10 14 17 19 20 21 22 24 26]
[ 0  3  5  8  9 10 11 12 14 15 16 17 19 20 21 22 24 26 28 29] [ 1  2  4  6  7 13 18 23 25 27]
[ 1  2  3  4  6  7 10 13 14 17 18 19 20 21 22 23 24 25 26 27] [ 0  5  8  9 11 12 15 16 28 29]
repetition 3 with seed 2
[ 2  4  5  7  8 10 11 12 13 15 16 17 18 22 24 25 26 27 28 29] [ 0  1  3  6  9 14 19 20 21 23]
[ 0  1  3  6  7  8  9 11 13 14 15 18 19 20 21 22 23 24 26 29] [ 2  4  5 10 12 16 17 25 27 28]
[ 0  1  2  3  4  5  6  9 10 12 14 16 17 19 20 21 23 25 27 28] [ 7  8 11 13 15 18 22 24 26 29]
